# Towards a software engineering agent that operates on ansible repos

In [1]:
import os

from se_agent.llm.api import call_llm_for_task
from se_agent.llm.model_configuration_manager import TaskName

In [2]:
message_template = """
Your are a DevOps Assistant. You understand Ansible roles and playbooks. You understand the basic structure of Ansible playbooks and roles. E.g., you know that Ansible roles follow the following folder structure:

- defaults: contains default variables for the role.
- files: contains static files (e.g., configuration files, scripts) that will be copied over to the target host(s) as-is.
- handlers: contains handler tasks. Handlers are typically triggered by a "notify" directive within tasks. They run once, at the end of a play, and only if they have been notified.
- meta: contains metadata about the role, such as role dependencies (other roles it relies on), supported platforms, and other descriptive information used by Ansible Galaxy.
- tasks: contains the main tasks and their sequence that the role will execute.
- templates: contains Jinja2 templates that will be rendered and copied over to the target host(s).
- vars: contains variables that will be used by the role.

You also understand the basic structure of Ansible playbooks. E.g., you know that an Ansible playbook is a YAML file that contains a list of plays. Each play is a set of tasks that will be executed on a set of hosts. Each task is a call to an Ansible module. You also know that a playbook can contain variables, handlers, and roles.

Please understand the following and generate a summary of semantic understanding for the file. Limit the summary to 100 words. You do not have to tell me that you've limited the summary to 100 words. Nor should you ask if I'd like you to help with anything else.

File Path: {file_path}

```{file_type}
{file_content}
```
"""

In [3]:
def generate_semantic_summary(file_path, component):
    extension_map = {
        "yml": "yaml",
        "j2": "jinja2"
    }
    file_type = extension_map.get(file_path.split(".")[-1], None)
    if file_type is None:
        print(f"Skipping file type for {file_path}")
        return None

    file = {
        "component": component,
        "path": file_path,
        "type": file_type
    }
    with open(file_path, "r") as f:
        file["content"] = f.read()

    try:
        semantic_summary = call_llm_for_task(
            task_name=TaskName.GENERATE_CODE_SUMMARY,
            messages=[
                {
                    "role": "system",
                    "content": message_template.format(file_component=file["component"], file_path=file["path"], file_type=file["type"], file_content=file["content"])
                }
            ]
        ).content
        print(f"Generated semantic summary for {file_path}")
        return semantic_summary
    except Exception as e:
        print(f"Error generating semantic summary for {file_path}: {e}")
        return None

In [4]:
# Ordered list of common role subdirectories and their descriptions
ROLE_COMPONENTS = [
    ("defaults", "Here are the variable defaults"),
    ("files", "Static files (e.g., config files, scripts) that will be copied as-is to the target host(s)."),
    ("handlers", "Contains 'handler' tasks, typically triggered by 'notify' within tasks. They run at the end of a play if notified."),
    ("meta", "Houses metadata about the role (dependencies, supported platforms, etc.)."),
    ("tasks", "Core of the role: the main sequence of tasks to be performed. May include other task files."),
    ("templates", "Jinja2 templates, which can dynamically insert variables/data before placement on the target host(s)."),
    ("tests", "Contains automated tests (e.g., Molecule) and sample playbooks for verifying role functionality."),
    ("vars", "Defines higher-priority variables than defaults. Typically shouldn't change frequently.")
]

def generate_role_documentation(repo_path, role_name):
    """
    Given:
      - repo_path: Path to the root of your Ansible repository
      - role_name: Name of the role (and possibly the top-level playbook)

    This function:
      1) Checks if there's a top-level playbook named {role_name}.yml in repo_path
      2) Documents all standard role subdirectories found in repo_path/role_name
    """
    # Full path to the role directory
    role_path = os.path.join(repo_path, role_name)

    # Start building our documentation as a list of lines
    docs_lines = []

    # 1. Document the main playbook (if it exists in the root and named <role_name>.yml)
    main_playbook_path = os.path.join(repo_path, f"{role_name}.yml")
    if os.path.isfile(main_playbook_path):
        docs_lines.append(f"# {role_name}")
        semantic_summary = generate_semantic_summary(main_playbook_path, "")
        if semantic_summary:
            docs_lines.append(semantic_summary)
        docs_lines.append("")  # Blank line

    # 2. Document the role directory structure
    for component, description in ROLE_COMPONENTS:
        subdir_path = os.path.join(role_path, component)
        if os.path.isdir(subdir_path):
            # Add the heading and descriptive text
            docs_lines.append(f"## {component}")
            docs_lines.append(description)
            docs_lines.append("")

            # Iterate over files in the subdirectory
            files_in_subdir = sorted(os.listdir(subdir_path))
            for f in files_in_subdir:
                file_path = os.path.join(subdir_path, f)

                # We only want to list actual files, not directories
                if os.path.isfile(file_path):
                    # Here we'll use our agent to generate a semantic summary for the file.
                    semantic_summary = generate_semantic_summary(file_path, component)
                    if semantic_summary:
                        docs_lines.append(f"### {f}")
                        docs_lines.append(f"{semantic_summary}")
                        docs_lines.append("")

    # Join all lines with newlines
    return "\n".join(docs_lines)

In [5]:
repo_path = "/Users/pdhoolia/ansible/awx_client"
role_name = "awx_client"

markdown_doc = generate_role_documentation(repo_path=repo_path, role_name=role_name)

# Let's write the documentation to a file
with open(f"{role_name}_documentation.md", "w") as f:
    f.write(markdown_doc)

Generated semantic summary for /Users/pdhoolia/ansible/awx_client/awx_client.yml
Skipping file type for /Users/pdhoolia/ansible/awx_client/awx_client/files/IBM-caintermediatecert.pem
Skipping file type for /Users/pdhoolia/ansible/awx_client/awx_client/files/IBM-carootcert.pem
Generated semantic summary for /Users/pdhoolia/ansible/awx_client/awx_client/handlers/main.yml
Generated semantic summary for /Users/pdhoolia/ansible/awx_client/awx_client/meta/main.yml
Generated semantic summary for /Users/pdhoolia/ansible/awx_client/awx_client/tasks/cleanup.yml
Generated semantic summary for /Users/pdhoolia/ansible/awx_client/awx_client/tasks/ibm_ca_certs.yml
Generated semantic summary for /Users/pdhoolia/ansible/awx_client/awx_client/tasks/main.yml
Generated semantic summary for /Users/pdhoolia/ansible/awx_client/awx_client/tasks/pam_access.yml
Generated semantic summary for /Users/pdhoolia/ansible/awx_client/awx_client/tasks/sanity.yml
Generated semantic summary for /Users/pdhoolia/ansible/awx